In [1]:
using Pkg; Pkg.activate("/home/dhairyagandhi96/modelzoo/model-zoo/script/../text/phonemes"); Pkg.instantiate(); Pkg.add(PackageSpec(name="CUDAdrv", rev = "master")); Pkg.add(PackageSpec(name="CUDAnative", rev = "master")); Pkg.add(PackageSpec(name="CuArrays", rev = "master")); Pkg.add(PackageSpec(name="Flux", rev = "master"))

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating git-repo `https://github.com/JuliaGPU/CUDAdrv.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/text/phonemes/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/text/phonemes/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/JuliaGPU/CUDAnative.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/text/phonemes/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/text/phonemes/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/JuliaGPU/CuArrays.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/text/phonemes/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/text/phonemes/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/FluxML/Flux.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/text/phon

Based on https://arxiv.org/abs/1409.0473

In [2]:
using Flux: flip, crossentropy, reset!, throttle

include("0-data.jl")

Nin = length(alphabet)
Nh = 30 # size of hidden layer

30

A recurrent model which takes a token and returns a context-dependent
annotation.

In [3]:
forward  = LSTM(Nin, Nh÷2)
backward = LSTM(Nin, Nh÷2)
encode(tokens) = vcat.(forward.(tokens), flip(backward, tokens))

alignnet = Dense(2Nh, 1)
align(s, t) = alignnet(vcat(t, s .* trues(1, size(t, 2))))

align (generic function with 1 method)

A recurrent model which takes a sequence of annotations, attends, and returns
a predicted output token.

In [4]:
recur   = LSTM(Nh+length(phones), Nh)
toalpha = Dense(Nh, length(phones))

function asoftmax(xs)
  xs = [exp.(x) for x in xs]
  s = sum(xs)
  return [x ./ s for x in xs]
end

function decode1(tokens, phone)
  weights = asoftmax([align(recur.state[2], t) for t in tokens])
  context = sum(map((a, b) -> a .* b, weights, tokens))
  y = recur(vcat(Float32.(phone), context))
  return softmax(toalpha(y))
end

decode(tokens, phones) = [decode1(tokens, phone) for phone in phones]

decode (generic function with 1 method)

The full model

In [5]:
state = (forward, backward, alignnet, recur, toalpha)

function model(x, y)
  ŷ = decode(encode(x), y)
  reset!(state)
  return ŷ
end

loss(x, yo, y) = sum(crossentropy.(model(x, yo), y))

evalcb = () -> @show loss(data[500]...)
opt = ADAM(params(state))

Flux.train!(loss, data, opt, cb = throttle(evalcb, 10))

┌ Warning: ADAM(params) is deprecated; use ADAM(η::Float64) instead
│   caller = top-level scope at none:0
└ @ Core none:0
┌ Warning: train!(loss, data, opt) is deprecated; use train!(loss, params, data, opt) instead
│   caller = ip:0x0
└ @ Core :-1
loss(data[500]...) = 31.0931f0 (tracked)
loss(data[500]...) = 18.719795f0 (tracked)
loss(data[500]...) = 16.85845f0 (tracked)
loss(data[500]...) = 15.588554f0 (tracked)
loss(data[500]...) = 15.574289f0 (tracked)
loss(data[500]...) = 14.420161f0 (tracked)
loss(data[500]...) = 13.901421f0 (tracked)
loss(data[500]...) = 14.280205f0 (tracked)
loss(data[500]...) = 13.549485f0 (tracked)
loss(data[500]...) = 14.676176f0 (tracked)
loss(data[500]...) = 14.110245f0 (tracked)
loss(data[500]...) = 16.086271f0 (tracked)
loss(data[500]...) = 15.810038f0 (tracked)
loss(data[500]...) = 17.4217f0 (tracked)
loss(data[500]...) = 18.87833f0 (tracked)
loss(data[500]...) = 22.803078f0 (tracked)


Prediction

In [6]:
using StatsBase: wsample

function predict(s)
  ts = encode(tokenise(s, alphabet))
  ps = Any[:start]
  for i = 1:50
    dist = decode1(ts, onehot(ps[end], phones))
    next = wsample(phones, vec(Tracker.data(dist)))
    next == :end && break
    push!(ps, next)
  end
  return ps[2:end]
end

predict("PHYLOGENY")

15-element Array{Any,1}:
 :M  
 :EY2
 :T  
 :R  
 :IH2
 :S  
 :T  
 :AH0
 :S  
 :EY1
 :SH 
 :AH0
 :NG 
 :L  
 :IH0